In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# -*- coding: utf-8 -*-
"""Final (Optimization).ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1_GP6hhj2Lauzt5OD4G1d08lL2XaBlY8A
"""

!pip install scikit-fuzzy

##mfDerivs.py
import numpy as np


def partial_dMF(x, mf_definition, partial_parameter):
    """Calculates the partial derivative of a membership function at a point x.



    Parameters
    ------


    Returns
    ------

    """
    mf_name = mf_definition[0]

    if mf_name == 'gaussmf':

        sigma = mf_definition[1]['sigma']
        mean = mf_definition[1]['mean']

        if partial_parameter == 'sigma':
            result = (2./sigma**3) * np.exp(-(((x-mean)**2)/(sigma)**2))*(x-mean)**2
        elif partial_parameter == 'mean':
            result = (2./sigma**2) * np.exp(-(((x-mean)**2)/(sigma)**2))*(x-mean)

    elif mf_name == 'gbellmf':

        a = mf_definition[1]['a']
        b = mf_definition[1]['b']
        c = mf_definition[1]['c']

        if partial_parameter == 'a':
            result = (2. * b * np.power((c-x),2) * np.power(np.absolute((c-x)/a), ((2 * b) - 2))) / \
                (np.power(a, 3) * np.power((np.power(np.absolute((c-x)/a),(2*b)) + 1), 2))
        elif partial_parameter == 'b':
            result = -1 * (2 * np.power(np.absolute((c-x)/a), (2 * b)) * np.log(np.absolute((c-x)/a))) / \
                (np.power((np.power(np.absolute((c-x)/a), (2 * b)) + 1), 2))
        elif partial_parameter == 'c':
            result = (2. * b * (c-x) * np.power(np.absolute((c-x)/a), ((2 * b) - 2))) / \
                (np.power(a, 2) * np.power((np.power(np.absolute((c-x)/a),(2*b)) + 1), 2))

    elif mf_name == 'sigmf':

        b = mf_definition[1]['b']
        c = mf_definition[1]['c']

        if partial_parameter == 'b':
            result = -1 * (c * np.exp(c * (b + x))) / \
                np.power((np.exp(b*c) + np.exp(c*x)), 2)
        elif partial_parameter == 'c':
            result = ((x - b) * np.exp(c * (x - b))) / \
                np.power((np.exp(c * (x - c))) + 1, 2)


    return result

##membershipfunction.py
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 31 15:41:58 2014

@author: tim.meggs
"""

from skfuzzy import gaussmf, gbellmf, sigmf

class MemFuncs:
    'Common base class for all employees'
    funcDict = {'gaussmf': gaussmf, 'gbellmf': gbellmf, 'sigmf': sigmf}


    def __init__(self, MFList):
        self.MFList = MFList

    def evaluateMF(self, rowInput):
        if len(rowInput) != len(self.MFList):
            print("Number of variables does not match number of rule sets")

        return [[self.funcDict[self.MFList[i][k][0]](rowInput[i],**self.MFList[i][k][1]) for k in range(len(self.MFList[i]))] for i in range(len(rowInput))]

##afnis.py
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 03 07:30:34 2014

@author: tim.meggs
"""
import itertools
import numpy as np

import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './membership')

import copy

class ANFIS:
    """Class to implement an Adaptive Network Fuzzy Inference System: ANFIS"

    Attributes:
        X
        Y
        XLen
        memClass
        memFuncs
        memFuncsByVariable
        rules
        consequents
        errors
        memFuncsHomo
        trainingType


    """

    def __init__(self, X, Y, memFunction):
        self.X = np.array(copy.copy(X))
        self.Y = np.array(copy.copy(Y))
        self.XLen = len(self.X)
        self.memClass = copy.deepcopy(memFunction)
        self.memFuncs = self.memClass.MFList
        self.memFuncsByVariable = [[x for x in range(len(self.memFuncs[z]))] for z in range(len(self.memFuncs))]
        self.rules = np.array(list(itertools.product(*self.memFuncsByVariable)))
        self.consequents = np.empty(self.Y.ndim * len(self.rules) * (self.X.shape[1] + 1))
        self.consequents.fill(0)
        self.errors = np.empty(0)
        self.memFuncsHomo = all(len(i)==len(self.memFuncsByVariable[0]) for i in self.memFuncsByVariable)
        self.trainingType = 'Not trained yet'

    def LSE(self, A, B, initialGamma = 1000.):
        coeffMat = A
        rhsMat = B
        S = np.eye(coeffMat.shape[1])*initialGamma
        x = np.zeros((coeffMat.shape[1],1)) # need to correct for multi-dim B
        for i in range(len(coeffMat[:,0])):
            a = coeffMat[i,:]
            b = np.array(rhsMat[i])
            S = S - (np.array(np.dot(np.dot(np.dot(S,np.matrix(a).transpose()),np.matrix(a)),S)))/(1+(np.dot(np.dot(S,a),a)))
            x = x + (np.dot(S,np.dot(np.matrix(a).transpose(),(np.matrix(b)-np.dot(np.matrix(a),x)))))
        return x

    def trainHybridJangOffLine(self, epochs=5, tolerance=1e-5, initialGamma=1000, k=0.01):

        self.trainingType = 'trainHybridJangOffLine'
        convergence = False
        epoch = 1

        while (epoch < epochs) and (convergence is not True):

            #layer four: forward pass
            [layerFour, wSum, w] = forwardHalfPass(self, self.X)

            #layer five: least squares estimate
            layerFive = np.array(self.LSE(layerFour,self.Y,initialGamma))
            self.consequents = layerFive
            layerFive = np.dot(layerFour,layerFive)

            #error
            error = np.sum((self.Y-layerFive.T)**2)
            print('current error: '+ str(error))
            average_error = np.average(np.absolute(self.Y-layerFive.T))
            self.errors = np.append(self.errors,error)

            if len(self.errors) != 0:
                if self.errors[len(self.errors)-1] < tolerance:
                    convergence = True

            # back propagation
            if convergence is not True:
                cols = range(len(self.X[0,:]))
                dE_dAlpha = list(backprop(self, colX, cols, wSum, w, layerFive) for colX in range(self.X.shape[1]))


            if len(self.errors) >= 4:
                if (self.errors[-4] > self.errors[-3] > self.errors[-2] > self.errors[-1]):
                    k = k * 1.1

            if len(self.errors) >= 5:
                if (self.errors[-1] < self.errors[-2]) and (self.errors[-3] < self.errors[-2]) and (self.errors[-3] < self.errors[-4]) and (self.errors[-5] > self.errors[-4]):
                    k = k * 0.9

            ## handling of variables with a different number of MFs
            t = []
            for x in range(len(dE_dAlpha)):
                for y in range(len(dE_dAlpha[x])):
                    for z in range(len(dE_dAlpha[x][y])):
                        t.append(dE_dAlpha[x][y][z])

            eta = k / np.abs(np.sum(t))

            if(np.isinf(eta)):
                eta = k

            ## handling of variables with a different number of MFs
            dAlpha = copy.deepcopy(dE_dAlpha)
            if not(self.memFuncsHomo):
                for x in range(len(dE_dAlpha)):
                    for y in range(len(dE_dAlpha[x])):
                        for z in range(len(dE_dAlpha[x][y])):
                            dAlpha[x][y][z] = -eta * dE_dAlpha[x][y][z]
            else:
                dAlpha = -eta * np.array(dE_dAlpha)


            for varsWithMemFuncs in range(len(self.memFuncs)):
                for MFs in range(len(self.memFuncsByVariable[varsWithMemFuncs])):
                    paramList = sorted(self.memFuncs[varsWithMemFuncs][MFs][1])
                    for param in range(len(paramList)):
                        self.memFuncs[varsWithMemFuncs][MFs][1][paramList[param]] = self.memFuncs[varsWithMemFuncs][MFs][1][paramList[param]] + dAlpha[varsWithMemFuncs][MFs][param]
            epoch = epoch + 1


        self.fittedValues = predict(self,self.X)
        self.residuals = self.Y - self.fittedValues[:,0]

        return self.fittedValues


    def plotErrors(self):
        if self.trainingType == 'Not trained yet':
            print(self.trainingType)
        else:
            import matplotlib.pyplot as plt
            plt.plot(range(len(self.errors)),self.errors,'ro', label='errors')
            plt.ylabel('error')
            plt.xlabel('epoch')
            plt.show()

    def plotMF(self, x, inputVar):
        import matplotlib.pyplot as plt
        from skfuzzy import gaussmf, gbellmf, sigmf

        for mf in range(len(self.memFuncs[inputVar])):
            if self.memFuncs[inputVar][mf][0] == 'gaussmf':
                y = gaussmf(x,**self.memClass.MFList[inputVar][mf][1])
            elif self.memFuncs[inputVar][mf][0] == 'gbellmf':
                y = gbellmf(x,**self.memClass.MFList[inputVar][mf][1])
            elif self.memFuncs[inputVar][mf][0] == 'sigmf':
                y = sigmf(x,**self.memClass.MFList[inputVar][mf][1])

            plt.plot(x,y,'r')

        plt.show()

    def plotResults(self):
        if self.trainingType == 'Not trained yet':
            print(self.trainingType)
        else:
            import matplotlib.pyplot as plt
            plt.plot(range(len(self.fittedValues)),self.fittedValues,'r', label='trained')
            plt.plot(range(len(self.Y)),self.Y,'b', label='original')
            plt.legend(loc='upper left')
            plt.show()



def forwardHalfPass(ANFISObj, Xs):
    layerFour = np.empty(0,)
    wSum = []

    for pattern in range(len(Xs[:,0])):
        #layer one
        layerOne = ANFISObj.memClass.evaluateMF(Xs[pattern,:])

        #layer two
        miAlloc = [[layerOne[x][ANFISObj.rules[row][x]] for x in range(len(ANFISObj.rules[0]))] for row in range(len(ANFISObj.rules))]
        layerTwo = np.array([np.product(x) for x in miAlloc]).T
        if pattern == 0:
            w = layerTwo
        else:
            w = np.vstack((w,layerTwo))

        #layer three
        wSum.append(np.sum(layerTwo))
        if pattern == 0:
            wNormalized = layerTwo/wSum[pattern]
        else:
            wNormalized = np.vstack((wNormalized,layerTwo/wSum[pattern]))

        #prep for layer four (bit of a hack)
        layerThree = layerTwo/wSum[pattern]
        rowHolder = np.concatenate([x*np.append(Xs[pattern,:],1) for x in layerThree])
        layerFour = np.append(layerFour,rowHolder)

    w = w.T
    wNormalized = wNormalized.T

    layerFour = np.array(np.array_split(layerFour,pattern + 1))

    return layerFour, wSum, w


def backprop(ANFISObj, columnX, columns, theWSum, theW, theLayerFive):

    paramGrp = [0]* len(ANFISObj.memFuncs[columnX])
    for MF in range(len(ANFISObj.memFuncs[columnX])):

        parameters = np.empty(len(ANFISObj.memFuncs[columnX][MF][1]))
        timesThru = 0
        for alpha in sorted(ANFISObj.memFuncs[columnX][MF][1].keys()):

            bucket3 = np.empty(len(ANFISObj.X))
            for rowX in range(len(ANFISObj.X)):
                varToTest = ANFISObj.X[rowX,columnX]
                tmpRow = np.empty(len(ANFISObj.memFuncs))
                tmpRow.fill(varToTest)

                bucket2 = np.empty(ANFISObj.Y.ndim)
                for colY in range(ANFISObj.Y.ndim):

                    rulesWithAlpha = np.array(np.where(ANFISObj.rules[:,columnX]==MF))[0]
                    adjCols = np.delete(columns,columnX)

                    senSit = partial_dMF(ANFISObj.X[rowX,columnX],ANFISObj.memFuncs[columnX][MF],alpha)
                    # produces d_ruleOutput/d_parameterWithinMF
                    dW_dAplha = senSit * np.array([np.prod([ANFISObj.memClass.evaluateMF(tmpRow)[c][ANFISObj.rules[r][c]] for c in adjCols]) for r in rulesWithAlpha])

                    bucket1 = np.empty(len(ANFISObj.rules[:,0]))
                    for consequent in range(len(ANFISObj.rules[:,0])):
                        fConsequent = np.dot(np.append(ANFISObj.X[rowX,:],1.),ANFISObj.consequents[((ANFISObj.X.shape[1] + 1) * consequent):(((ANFISObj.X.shape[1] + 1) * consequent) + (ANFISObj.X.shape[1] + 1)),colY])
                        acum = 0
                        if consequent in rulesWithAlpha:
                            acum = dW_dAplha[np.where(rulesWithAlpha==consequent)] * theWSum[rowX]

                        acum = acum - theW[consequent,rowX] * np.sum(dW_dAplha)
                        acum = acum / theWSum[rowX]**2
                        bucket1[consequent] = fConsequent * acum

                    sum1 = np.sum(bucket1)

                    if ANFISObj.Y.ndim == 1:
                        bucket2[colY] = sum1 * (ANFISObj.Y[rowX]-theLayerFive[rowX,colY])*(-2)
                    else:
                        bucket2[colY] = sum1 * (ANFISObj.Y[rowX,colY]-theLayerFive[rowX,colY])*(-2)

                sum2 = np.sum(bucket2)
                bucket3[rowX] = sum2

            sum3 = np.sum(bucket3)
            parameters[timesThru] = sum3
            timesThru = timesThru + 1

        paramGrp[MF] = parameters

    return paramGrp


def predict(ANFISObj, varsToTest):

    [layerFour, wSum, w] = forwardHalfPass(ANFISObj, varsToTest)

    #layer five
    layerFive = np.dot(layerFour,ANFISObj.consequents)

    return layerFive


if __name__ == "__main__":
    print("I am main!")

#from google.colab import files
#uploaded = files.upload()

##svr_anfis.py
##IMPORTING REQUIRED PACKAGES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import sys
from sklearn.metrics import mean_absolute_error as mae
import math
from sklearn.model_selection import GridSearchCV


##HYPERPARAMETERS
epochs_anfis = 50

##LOAD ORIGINIAL DATA AND VISUALIZE THEM
dataset = pd.read_csv('../input/well-water-data/abkhan_truncated4.csv') 
y = dataset.values[:, -1].ravel()
print(y)
x1 = np.array(range(y.shape[0]))
x1 = x1.reshape(len(x1), 1)
x = dataset.values[0:178 , 25:26]
x1_train = dataset.values[0:140 , 0]
x_train = dataset.values[0:140 , 25:26]
y_train = dataset.values[0:140 , -1]
y_train = y_train.reshape(y_train.shape[0], 1)
x1_test= dataset.values[140:178 , 0]
x_test = dataset.values[140:178 , 25:26]
y_test = dataset.values[140:178 , -1]
y_test = y_test.reshape(y_test.shape[0], 1)
#plt.plot(x)                                           # dispaly original data
#plt.xlabel('Time (month)')
#plt.ylabel('Ground Water Level (m)')
#plt.title('Original Data')
#plt.show()

#GridSearch
param = {'kernel' : ('poly', 'rbf', 'sigmoid'),'C' : [1,5,10],'degree' : [3,8],'coef0' : [0.01,10,0.5],'gamma' : ('auto','scale')},

regressior = SVR(),

regressior = GridSearchCV(regressior,param,cv=5)

#grids.fit(Xtrain,ytrain)

##IMPLEMENTING SVR (%80 of Data fo Training)
#regressior = SVR()
for k in range(1,1000,10):
    regressior = SVR(C=k)
    regressior1=SVR()
    regressior.fit(x_train, y_train)
    regressior1.fit(x_train, y_train)
    confidence = regressior.score(x_test, y_test)
    print(k,confidence)
y_pred_svr = regressior.predict(x)
y_pred_svr1 = regressior1.predict(x)
y_pred1_svr = regressior.predict(x_train)
y_pred1_svr1 = regressior1.predict(x_train)
y_pred2_svr = regressior.predict(x_test)
y_pred2_svr1 = regressior1.predict(x_test)

plt.title('SVR Test')
plt.scatter(x1_test, y_test, color='red', label='Original Data')
plt.plot(x1_test,y_pred2_svr, color='blue', label='SVR', linewidth=3)
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()

plt.title('SVR Total Results')
plt.scatter(x1, y, color='red', label='Original Data')
plt.plot(x1,y_pred_svr1, color='Blue', label='SVR', linewidth=3)
plt.plot(x1,y_pred_svr, color='Orange', label='Optimal SVR', linewidth=3)
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()

##PRINTING THE RESULTS OF SVR
print('==========================\nSVR Scores:\n') 
print('{:30} {}'.format('R2 for Total Data:', r2_score(y, y_pred_svr)))
print('{:30} {}'.format('R2 for Training Data:', r2_score(y_train, y_pred1_svr)))
print('{:30} {}'.format('R2 for Test Data:', r2_score(y_test, y_pred2_svr)))
print('{:30} {}'.format('MSE for Total Data:', mean_squared_error(y, y_pred_svr)))
print('{:30} {}'.format('MSE for Training Data:', mean_squared_error(y_train, y_pred1_svr)))
print('{:30} {}'.format('MSE for Test Data:', mean_squared_error(y_test, y_pred2_svr)))
print('{:30} {}'.format('MAE for Total Data:', mae(y, y_pred_svr)))
print('{:30} {}'.format('MAE for Training Data:', mae(y_train, y_pred1_svr)))
print('{:30} {}'.format('MAE for Test Data:', mae(y_test, y_pred2_svr)))
print('{:30} {}'.format('RMSE for Total Data:', math.sqrt(mean_squared_error(y, y_pred_svr))))
print('{:30} {}'.format('RMSE for Training Data:', math.sqrt(mean_squared_error(y_train, y_pred1_svr))))
print('{:30} {}'.format('RMSE for Test Data:', math.sqrt(mean_squared_error(y_test, y_pred2_svr))))
print('==========================')

##IMPLEMENTING ANFIS
sys.path.insert(1, './membership')

#Define membership functions
mf = [[['gaussmf',{'mean':1667.,'sigma':10.}],['gaussmf',{'mean':1672.,'sigma':20.}],['gaussmf',{'mean':1677.,'sigma':40.}]]]

mfc = MemFuncs(mf)
anf = ANFIS(x_train, y_train.ravel(), mfc)
anf.trainHybridJangOffLine(epochs=epochs_anfis)
print(round(anf.consequents[-1][0],6))
print(round(anf.consequents[-2][0],6))
print(round(anf.fittedValues[9][0],6))
if round(anf.consequents[-1][0],6) == -5.275538 and round(anf.consequents[-2][0],6) == -1.990703 and round(anf.fittedValues[9][0],6) == 0.002249:
	print('test is good')
#anf.plotErrors()
##anf.plotResults()

y_pred_anfis = predict(anf, x)
y_pred1_anfis = predict(anf, x_train)
y_pred2_anfis = predict(anf, x_test)

#plt.plot(y_pred_anfis)
#plt.show()

plt.title('ANFIS Train')
plt.scatter(x1_train, y_train, color='red', label='Original Data')
plt.plot(x1_train, y_pred1_anfis, color='blue', label='ANFIS', linewidth=3)
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()

plt.title('ANFIS Test')
plt.scatter(x1_test, y_test, color='red', label='Original Data')
plt.plot(x1_test,y_pred2_anfis, color='magenta', label='ANFIS', linewidth=3)
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()

plt.title('ANFIS Total Results')
plt.scatter(x1, y, color='red', label='Original Data')
plt.plot(x1,y_pred_anfis, color='magenta', label='ANFIS', linewidth=3)
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()
##Optimal ANFIS
#Define membership functions
mfOP = [[['gaussmf',{'mean':1667.,'sigma':7}],['gaussmf',{'mean':1672.,'sigma':7}],['gaussmf',{'mean':1677.,'sigma':7}]]]

mfcOP = MemFuncs(mfOP)
anfOP = ANFIS(x_train, y_train.ravel(), mfcOP)
anfOP.trainHybridJangOffLine(epochs=epochs_anfis)
print(round(anfOP.consequents[-1][0],6))
print(round(anfOP.consequents[-2][0],6))
print(round(anfOP.fittedValues[9][0],6))
if round(anfOP.consequents[-1][0],6) == -5.275538 and round(anfOP.consequents[-2][0],6) == -1.990703 and round(anfOP.fittedValues[9][0],6) == 0.002249:
	print('test is good')
#anf.plotErrors()
##anf.plotResults()

y_predOP_anfis = predict(anfOP, x)
y_pred1OP_anfis = predict(anfOP, x_train)
y_pred2OP_anfis = predict(anfOP, x_test)

#plt.plot(y_pred_anfis)
#plt.show()

plt.title('ANFIS OP Train')
plt.scatter(x1_train, y_train, color='red', label='Original Data')
plt.plot(x1_train, y_pred1OP_anfis, color='blue', label='ANFIS', linewidth=3)
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()

plt.title('ANFIS OP Test')
plt.scatter(x1_test, y_test, color='red', label='Original Data')
plt.plot(x1_test,y_pred2OP_anfis, color='magenta', label='ANFIS', linewidth=3)
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()

plt.title('ANFIS OP Total Results')
plt.scatter(x1, y, color='red', label='Original Data')
plt.plot(x1,y_predOP_anfis, color='magenta', label='ANFIS', linewidth=3)
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()

##PRINTING THE RESULTS OF ANFIS
print('==========================\nANFIS Scores:\n') 
print('{:30} {}'.format('R2 for Total Data OP:', r2_score(y, y_predOP_anfis)))
print('{:30} {}'.format('R2 for Training Data:', r2_score(y_train, y_pred1OP_anfis)))
print('{:30} {}'.format('R2 for Test Data:', r2_score(y_test, y_pred2OP_anfis)))
print('{:30} {}'.format('MSE for Total Data:', mean_squared_error(y, y_predOP_anfis)))
print('{:30} {}'.format('MSE for Training Data:', mean_squared_error(y_train, y_pred1OP_anfis)))
print('{:30} {}'.format('MSE for Test Data:', mean_squared_error(y_test, y_pred2OP_anfis)))
print('{:30} {}'.format('MAE for Total Data:', mae(y, y_predOP_anfis)))
print('{:30} {}'.format('MAE for Training Data:', mae(y_train, y_pred1OP_anfis)))
print('{:30} {}'.format('MAE for Test Data:', mae(y_test, y_pred2OP_anfis)))
print('{:30} {}'.format('RMSE for Total Data:', math.sqrt(mean_squared_error(y, y_predOP_anfis))))
print('{:30} {}'.format('RMSE for Training Data:', math.sqrt(mean_squared_error(y_train, y_pred1OP_anfis))))
print('{:30} {}'.format('RMSE for Test Data:', math.sqrt(mean_squared_error(y_test, y_pred2OP_anfis))))

###########################################################33
plt.title('ANFIS and SVR comparision')
plt.scatter(x1, y, color='red', label='Original Data')
plt.plot(x1, y_pred_anfis, color='magenta', label='ANFIS', linewidth=3)
plt.plot(x1, y_pred_svr, color='blue', label='SVR', linewidth=3)
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()

##PRINTING THE RESULTS OF ANFIS
print('==========================\nANFIS Scores:\n') 
print('{:30} {}'.format('R2 for Total Data:', r2_score(y, y_pred_anfis)))
print('{:30} {}'.format('R2 for Training Data:', r2_score(y_train, y_pred1_anfis)))
print('{:30} {}'.format('R2 for Test Data:', r2_score(y_test, y_pred2_anfis)))
print('{:30} {}'.format('MSE for Total Data:', mean_squared_error(y, y_pred_anfis)))
print('{:30} {}'.format('MSE for Training Data:', mean_squared_error(y_train, y_pred1_anfis)))
print('{:30} {}'.format('MSE for Test Data:', mean_squared_error(y_test, y_pred2_anfis)))
print('{:30} {}'.format('MAE for Total Data:', mae(y, y_pred_anfis)))
print('{:30} {}'.format('MAE for Training Data:', mae(y_train, y_pred1_anfis)))
print('{:30} {}'.format('MAE for Test Data:', mae(y_test, y_pred2_anfis)))
print('{:30} {}'.format('RMSE for Total Data:', math.sqrt(mean_squared_error(y, y_pred_anfis))))
print('{:30} {}'.format('RMSE for Training Data:', math.sqrt(mean_squared_error(y_train, y_pred1_anfis))))
print('{:30} {}'.format('RMSE for Test Data:', math.sqrt(mean_squared_error(y_test, y_pred2_anfis))))

import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras import backend
np.random.seed(1234)
tf.random.set_seed(1234)

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error as mae
import math

#from google.colab import files
#uploaded = files.upload()
!pip install xlrd
#!pip install openpyxl
import xlrd
data = pd.read_excel('../input/d/hamedetezadi/well-water-level-meter/data2.xlsx')
labeled_data = data[data['Y'].notna()]
labeled_data

labeled_data

"""# Data pre-peration

```
# This is formatted as code
```


"""

Xn = labeled_data[[x for x in data.columns if x != 'Y']]
Xn = Xn.fillna(labeled_data.mean())
Xn  = Xn[[x for x in Xn.columns][2:26]]
Yn = labeled_data['Y']
#X_train1, X_test1, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=40)
x_train_mlp, x_test_mlp, y_train_mlp, y_test_mlp = train_test_split(Xn, Yn, test_size=0.2, random_state=40)

import matplotlib.pyplot as plt
plt.plot(Yn)
plt.ylabel('Y value')
plt.show()
set([int(x) for x in Yn])

"""# The Model"""

def baseline_model(inp_size, neurons, x, y):
  # create model
  model = Sequential()
  model.add(Dense(neurons, input_dim=inp_size, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal'))
  # Compile model
  model.compile(loss='mean_squared_error', optimizer='adam')
  model.fit(x, y, epochs=100)
  return model

#for i in range(512,1024,512):
model = baseline_model(23, 512, x_train_mlp, y_train_mlp)

pred_train= model.predict(x_train_mlp)
print('Training RMSE:', np.sqrt(mean_squared_error(y_train_mlp,pred_train)))
print('Training MSE:', mean_squared_error(y_train_mlp,pred_train))
print('Training R2:', r2_score(y_train_mlp,pred_train))
print('Training MAE:', mae(y_train_mlp,pred_train))
pred_test= model.predict(x_test_mlp)
print('Test RMSE:', np.sqrt(mean_squared_error(y_test_mlp,pred_test)))
print('Test MSE:', mean_squared_error(y_test_mlp,pred_test))
print('Test R2:', r2_score(y_test_mlp,pred_test))
print('Test MAE:', mae(y_test_mlp,pred_test))
pred= model.predict(Xn)
print('Total RMSE:', np.sqrt(mean_squared_error(Yn,pred))) 
print('Total MSE:', mean_squared_error(Yn,pred)) 
print('Total R2:', r2_score(Yn,pred))
print('Total MAE:', mae(Yn,pred))

plt.title('MLP')
Yn=Yn.ravel()
xn = np.array(range(Yn.shape[0]))
xn = xn.reshape(len(xn), 1)
yn = Yn.reshape(Yn.shape[0], 1)
plt.plot(pred, color='green', label='MLP', linewidth=3)
plt.scatter(xn, yn, color='red', label='Original Data')
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()

plt.title('MLP and ANFIS comparision')
Yn=Yn.ravel()
xn = np.array(range(Yn.shape[0]))
xn = xn.reshape(len(xn), 1)
yn = Yn.reshape(Yn.shape[0], 1)
plt.plot(pred, color='green', label='MLP', linewidth=3)
plt.plot(x1,y_pred_anfis, color='magenta', label='ANFIS', linewidth=3)
plt.scatter(xn, yn, color='red', label='Original Data')
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()

"""# Forcasting"""

plt.title('ANFIS and SVR and MLP comparision')
#plt.scatter(xn, yn, color='red', label='Original Data')
plt.plot(x1, y_pred_svr, color='blue', label='SVR', linewidth=2)
plt.plot(x1, y_pred_anfis, color='magenta', label='ANFIS', linewidth=2)
plt.plot(pred, color='green', label='MLP', linewidth=2)
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()

plt.title('MLP and ANFIS and Optimal SVR comparision')
Yn=Yn.ravel()
xn = np.array(range(Yn.shape[0]))
xn = xn.reshape(len(xn), 1)
yn = Yn.reshape(Yn.shape[0], 1)
plt.plot(pred, color='green', label='MLP', linewidth=2)
plt.plot(x1,y_pred_anfis, color='magenta', label='ANFIS', linewidth=2)
plt.plot(x1,y_pred_svr, color='orange', label='Optimal SVR', linewidth=2)
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()

plt.title('SVR and Optimal SVR and ANFIS comparision')
plt.plot(x1,y_pred_svr1, color='Blue', label='SVR', linewidth=2)
plt.plot(x1,y_pred_svr, color='orange', label='Optimal SVR', linewidth=2)
plt.plot(x1,y_pred_anfis, color='magenta', label='ANFIS', linewidth=2)
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()

plt.title('Original Data and ANFIS and Optimal ANFIS')
plt.scatter(x1, y, color='red', label='Original Data')
plt.plot(x1,y_pred_anfis, color='magenta', label='ANFIS', linewidth=2)
plt.plot(x1,y_predOP_anfis, color='yellow', label='Optimal ANFIS', linewidth=2)
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()

plt.title('Original Data and Optimal SVR and Optimal ANFIS')
plt.scatter(x1, y, color='red', label='Original Data')
plt.plot(x1,y_pred_svr, color='orange', label='Optimal SVR', linewidth=2)
plt.plot(x1,y_predOP_anfis, color='yellow', label='Optimal ANFIS', linewidth=2)
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()

plt.title('MLP and Optimal SVR and Optimal ANFIS')
plt.plot(pred, color='green', label='MLP', linewidth=2)
plt.plot(x1,y_pred_svr, color='orange', label='Optimal SVR', linewidth=2)
plt.plot(x1,y_predOP_anfis, color='yellow', label='Optimal ANFIS', linewidth=2)
plt.legend()
plt.xlabel('Time (month)')
plt.ylabel('Ground Water Level (m)')
plt.show()